<a href="https://colab.research.google.com/github/PRaezUC3M/cohort_selection/blob/master/CNN_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is backed with github.
https://github.com/PRaezUC3M/cohort_selection/

**Mount the drive folder**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Configurartion of the experiments**
The different possible experiments have been automated. It is inspired in grid-search, and the parameters must be expressed in array-like style. You can find an example below.

Every possible example will be generated from the combination of the different parameters given.

In [0]:
import numpy as np
import tensorflow as tf

np.random.seed(1)
tf.set_random_seed(2)

In [0]:
# CNN Model
oversampling = [False]
load_embeddings = [True, False] # This defines if the embeddings are loaded from the wikipedia dataset
num_classes = [None] # Auto
embeddings_size = [200]
conv_size = [[128]] 
conv_filter = [[3]]
dropout = [0.45]
fnn_size = [[64, 13], [None]] 
batch_size = [32]

# VDCNN Model
oversampling = [False]
load_embeddings = [True, False] # This defines if the embeddings are loaded from the wikipedia dataset
num_classes = [None] # Auto
embeddings_size = [200]
conv_size = [[64, 64, 128, 128, 256, 256]]
conv_filter = [[10]]
dropout = [0.45]
fnn_size = [[256, 128, 128, 64], [None]]
batch_size = [32]



indexes = ["oversampling", "load_embeddings", "num_classes", "embeddings_size", "conv_size", "conv_filter", "dropout", "fnn_size", "batch_size"]
param   = [oversampling, load_embeddings, num_classes, embeddings_size, conv_size, conv_filter, dropout, fnn_size, batch_size]

In [0]:
import itertools

def combine_params(param, indexes):
  combinations = list(itertools.product(*param))
  param_combinations = [{k:v for k, v in zip(indexes, combination)}  for combination in combinations]
  for p in param_combinations:
    # The embeddings size must adapt to the embeddings loaded.
    if p["load_embeddings"]:
      p["embeddings_size"] = None
  return param_combinations

In [0]:
network_parameters = combine_params(param, indexes)

# **Load the train and test csv files**
This functions load the data and generate the different datasets from the given csv files. 

In [0]:
import pandas as pd
import numpy as np

SST_HOME='drive/My Drive/Colab Notebooks/CohortSelection/'
path_train=SST_HOME+'data/train/train.csv'
path_test=SST_HOME+'data/test/test.csv'

def load(path):
  
  df = pd.read_csv(path,header=0, delimiter="\t", quoting=3)

  categories=df.columns[2:];

  idFiles = df[['IDFILE']].as_matrix().tolist()

  texts = df[['TEXT']].as_matrix()
  X = [x[0].strip() for x in texts.tolist()]

  #we only keep the columns with the categories.
  Y = df.drop(['IDFILE', 'TEXT'], axis=1).as_matrix()


  print(path,'dataset loaded')
  
  return X, Y, categories, idFiles


train_x, train_y, CATEGORIES, _ = load(path_train)
test_x, test_y, _, IDFILES = load(path_test)

In [0]:
train_y[0].shape

#**Balancing the categories**
We can see that there are some categories way more present than others in our dataset. Let's sort them.

The categories are distributed in the following way


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns  

fig = plt.figure(figsize=(20,8))
(ax_train, ax_test) = fig.subplots(ncols=2, nrows=1)
g1 = sns.barplot(x=train_y.sum(axis=0), y=CATEGORIES, ax=ax_train)
ax_train.set_xlabel('Number of records')
g2 = sns.barplot(x=test_y.sum(axis=0), y=CATEGORIES, ax=ax_test)
ax_test.set_xlabel('Number of records')
g1.set_title("Criteria distribution on training dataset")
g2.set_title("Criteria distribution on testing dataset")

Now we can see the differences in a easier way, we will add weights to the different classes, based on how present they are in the dataset.

In [0]:
classes = np.add(sum(train_y), sum(test_y))
total_cases = sum(classes)
class_weight = [1 / (c/total_cases) for c in classes]

# **Tokenize the dataset and use the word embeddings**

This shows the set of different words ordered by frequency. *tokenizer.word_index*

The text is tokenized and cleaned of stop-words as well as punctuation signs. Also the numbers are substituted by a token "nmbr"

We crop the beggining of the examples because it is the date when they
were written down.

Finally, and after being tokenized, the different sentences are padded to match the maximum length.

max_length == 7813 it is really a huge vector because we have to pad
it afterwards, in order to get it into the CNN

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=stopwords.words('english')
# Define maximum word length
MAX_WORDS = 5000
# Defined from the dataset itself.
DATE_LENGTH = 23

In [0]:
# We are using this function to clean the test set
def tokenize_clean_text(text, tokenizer=None, max_length=None, 
                        max_words=MAX_WORDS, date_length=DATE_LENGTH):
  """
  This function is in charge of tokenizing the text it is given. It also cleans
  the text from stop-words, punctuation, and gives a special token to numbers.
  
  :param text: The texts to tokenize in a bidimensional python array.
  
  :returns: The tokenized and cleaned text in a bidimensional python array.
            The tokenizer used to preprocess the text.
            The maximum length used for padding.
  """  
  # Consider to stemm or lemmatize the text 
  
  cropped_date_text = [sentence[date_length:] for sentence in text]

  if max_length == None:
    # We get the maximum token length by splitting by spaces
    max_length = max([len(sentence[date_length:].split(" ")) for sentence in cropped_date_text])
    
  # We remove the numbers
  cropped_date_numbers_text = [" ".join([word if not word.isdigit() else ""
                                for word in sentence.split()])
                               for sentence in cropped_date_text]
  
  # Delete stopwords as well as every word less than 3 chars.
  cropped_date_numbers_stopw_text = [" ".join([word if not (word in stop_words or len(word) <= 3) else ""
                                      for word in sentence.split()])
                                     for sentence in cropped_date_numbers_text]
  
  if tokenizer is None:
    tokenizer = Tokenizer(num_words=MAX_WORDS) # They use 5k words too
    tokenizer.fit_on_texts(cropped_date_numbers_stopw_text)
  # We tokenize the sentences
  tokenized_text = tokenizer.texts_to_sequences(cropped_date_numbers_stopw_text)
  
  # Now we return the padded the sequences.
  return pad_sequences(tokenized_text, max_length), tokenizer, max_length
  

# **Oversampling**

As the dataset is very unbalanced, we must perform over-sampling (which is to increase the minority class(es)). The RandomOverSampler **class** allows us to over-sample minority classes by  picking samples at random with replacement. We need to install the package **imbalanced-learn**.


In [0]:
!pip3 install -U imbalanced-learn scikit-multilearn

In [0]:
from skmultilearn.problem_transform import LabelPowerset
from imblearn.over_sampling import RandomOverSampler

import seaborn as sns

def oversample(train_x, train_y):
  lp = LabelPowerset()
  ros = RandomOverSampler(random_state=42)

  # Applies the above stated multi-label (ML) to multi-class (MC) transformation.
  yt = lp.transform(train_y)
  
  x_resampled, y_resampled = ros.fit_sample(train_x_token, yt)

  # Inverts the ML-MC transformation to recreate the ML set
  y_resampled = lp.inverse_transform(y_resampled)
  y_resampled = np.asarray([i.toarray()[0] for i in y_resampled])
  
  fig = plt.figure(figsize=(10, 10))
  (ax_test, ax_train) = fig.subplots(ncols=2, nrows=1)
  g1 = sns.barplot(x=train_y.sum(axis=0), y=CATEGORIES, ax=ax_test)
  g2 = sns.barplot(x=y_resampled.sum(axis=0), y=CATEGORIES, ax=ax_train)
  g1.set_title("class distribution before resampling")
  g2.set_title("class distribution in training set after resampling")
  
  return x_resampled, y_resampled

# CNN Input

Here we take a maximum length of file by obtaining the accumulative density distribution of the training examples. This way we can see how many examples fall under certain threshold. Which will be fixed to 80% of the total examples. This length is 5000 words roughly.

In [0]:
cropped_date_text = [sentence[DATE_LENGTH:] for sentence in train_x]
lenghts = [len(sentence[DATE_LENGTH:].split(" ")) for sentence in cropped_date_text]
plt.hist(lenghts, cumulative=True)

In [0]:
# Thus, we initialize the maximum word length as
max_length = 5000

In [0]:
train_x_token, tokenizer, max_length = tokenize_clean_text(train_x, max_length=max_length)

if any([p["oversampling"] for p in network_parameters]):
  train_x_oversampled, train_y_oversampled = oversample(train_x_token, train_y)
  
test_x_token, _, _ = tokenize_clean_text(test_x, tokenizer, max_length)
assert len(train_x_token) == len(train_x)
assert len(test_x_token) == len(test_x)

# **TensorBoard**
TensorBoard is a great tool for DL visualization. It shows the evolution of metrics during the training phase, as well as the weights, distributions, and even the graph of the neural net. 

We will be using tensorboardcolab in order to run a 
TensorBoard instance. This will initialize a ngrok machine and launch TensorBoard for us to see. 

TensorBoard will be accesible by the url 

In [0]:
# We install tensorboard colab in case we don't have it already.
!pip install tensorboardcolab

In [0]:
import tensorboardcolab as tb

tbc=tb.TensorBoardColab()

# Load the embeddings

In this step, we load the word embeddings from the wikipedia, pubmed and PMC. Then we filter them adapting to the words we have in our dataset. This is, if the word appears in the pubmed we take its weights, which are set to 0 otherwise.

In [0]:
# seemingly gensim is not installed in google colab
!pip install gensim

In [0]:
from gensim.models.keyedvectors import KeyedVectors

PATH_W2V = "drive/My Drive/Colab Notebooks/CohortSelection/DL/word2vect/wikipedia-pubmed-and-PMC-w2v.bin"

In [0]:
def load_W2V_model(path):
    model = KeyedVectors.load_word2vec_format(path, binary=True)
    print("Loaded W2V model")
    return model
  
def generate_embedding_weigths(word_index, max_words, model):
  embedding_matrix = np.zeros((max_words, model.vector_size), dtype=np.float32)
  for word, i in word_index.items():
    if i >= max_words:
        break
    if word in model:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = model[word]
  return embedding_matrix

In [0]:
if any([p["load_embeddings"] for p in network_parameters]):
  # load the model
  if not 'model' in locals():
    model = load_W2V_model(PATH_W2V)
  # Generate the weights matrix
  embedding_matrix = generate_embedding_weigths(tokenizer.word_index, MAX_WORDS, model)
  # Set the embeddings size to the size of the embedding vector
  for p in network_parameters:
    if p["load_embeddings"]:
      p["embeddings_size"] = model.vector_size

In [0]:
network_parameters

#**Create a keras Embedding model**

In this section we will create a Keras CNN model, compile, and train it.

In this model we can decide to generate the different embeddings for the words by training the Embedding Keras layer, or simply use a pre-trained embeddings as the wikipedia ones for example.

Some information about the architecture of the net is shown bellow. 

In [0]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D
from keras.layers import MaxPooling1D, Dropout, Conv1D, Input
from keras.layers.merge import Concatenate, add
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model

In [0]:
# Generate the sequential baseline model
model_list = []
print(len(network_parameters))
for param in network_parameters:
  # oversampling, load_embeddings, num_classes, embeddings_size, conv_size, conv_filter, dropout, fnn_size, batch_size
  model_input = Input(shape=(max_length,))
  # Add the embeddings
  if param["load_embeddings"]:
    z = Embedding(MAX_WORDS, param["embeddings_size"], input_length=max_length, weights=[embedding_matrix], trainable=False)(model_input)
  else:
    z = Embedding(MAX_WORDS, param["embeddings_size"], input_length=max_length)(model_input)
    
  z = Dropout(param["dropout"])(z)
            
  # Add the cnn
  conv_blocks = []
  for filter_size in param["conv_filter"]:
    conv = None
    for i, cnn_layer in enumerate(param["conv_size"]):
      conv = Conv1D(cnn_layer, filter_size, padding='valid', activation='relu', strides=1)(z if conv is None else conv)
      if (i + 1) % 2 == 0:
        conv = MaxPooling1D(pool_size=filter_size)(conv)        
    
    conv_block = Flatten()(conv)
    conv_blocks.append(conv_block)         
   
  z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
            
  # Add the fnn
  fnn = None
  for fnn_layer in param["fnn_size"]:
    if fnn_layer is None:
      break
    fnn = Dense(fnn_layer)(z if fnn is None else fnn)
    fnn = Dropout(param["dropout"])(fnn)
  
  if param["num_classes"] is None:
    param["num_classes"] = len(train_y[0])
    
  fnn = Dense(param["num_classes"])(z if fnn is None else fnn)
  model_output = Activation('sigmoid')(fnn)

  net_model = Model(model_input, model_output)
  # Compile the model
  net_model.compile(optimizer="adam", loss="binary_crossentropy",
                    metrics=["acc"])

  # Print a summary of it.
  net_model.summary()
  plot_model(net_model, to_file=SST_HOME+"DL/models/"+'model_{}.png'.format(len(model_list)))
  model_list.append(net_model)

In [0]:
plot_model(net_model, to_file=SST_HOME+"DL/models/"+'model_{}.png'.format(len(model_list)))



In [0]:
import os
from keras.callbacks import TensorBoard
import tensorflow as tf

class TensorBoardColabCallback(TensorBoard):
    def __init__(self, tbc=None, write_graph=True, name=None, **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'

        if tbc is None:
            return

        log_dir = tbc.get_graph_path()

        training_log_dir = os.path.join(log_dir, 'training_{}'.format(name))
        super(TensorBoardColabCallback, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation_{}'.format(name))

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TensorBoardColabCallback, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}

        for name, value in val_logs.items():
            # print('val_logs:',epoch, name, value)
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TensorBoardColabCallback, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TensorBoardColabCallback, self).on_train_end(logs)
        self.val_writer.close()

tb.TensorBoardColabCallback = TensorBoardColabCallback

In [0]:
def define_callbacks(name):
  # Define the callbacks
  tbc_callback = tb.TensorBoardColabCallback(tbc, name=name)
   
  callbacks = [
      ReduceLROnPlateau(),
      EarlyStopping(patience=4),
      tbc_callback
  ]
  return callbacks

In [0]:
# Create an array of names
network_names = []
for p in network_parameters:
  name = "oversampling_{}_load_embeddings_t_{}_num_classes_{}_embeddings_size_{}_conv_size_{}_conv_filter_{}_dropout_{}_fnn_size_{}_batch_size_{}".format(
      p["oversampling"], p["load_embeddings"], p["num_classes"], p["embeddings_size"],
      p["conv_size"], p["conv_filter"], p["dropout"], p["fnn_size"], p["batch_size"])
  name = name.replace(" ", "").replace("[", "").replace("]", "").replace(",", "-")
  network_names.append(name)

## The models are trained

In [0]:
from keras.models import load_model
for i, net_model in enumerate(model_list):
  # Set a name for the model based on the tweaked parameters
  p = network_parameters[i]
  name = network_names[i]
  model_path = SST_HOME+"DL/models/" + name

  # If the model exists, don't compute it again.
  #if os.path.isfile(model_path):
  #  continue
    
  print("\n\n********************************************\n")    
  print(network_names[i])
  callbacks = define_callbacks(network_names[i])
  # Fit the model and extract its data
  if network_parameters[i]["oversampling"]:
    history = net_model.fit(train_x_oversampled, train_y_oversampled, epochs=30, batch_size=network_parameters[i]["batch_size"], 
                            class_weight=class_weight, callbacks=callbacks,
                            validation_split=0.25)
  else:
    history = net_model.fit(train_x_token, train_y, epochs=30, batch_size=network_parameters[i]["batch_size"], 
                            class_weight=class_weight, callbacks=callbacks,
                            validation_split=0.25)
      
  # And save the model
  net_model.save(model_path)
  
# To free memory from the gpu
from keras import backend as K
K.clear_session()

## The models are loaded


In [0]:
  from keras import backend as K

In [0]:
from sklearn.metrics import classification_report
predictions_list = []
for i, net_model in enumerate(model_list):
  print("\n\n********************************************\n")
  print(network_names[i])
  model_path = SST_HOME+"DL/models/" + network_names[i]
  try:
    net_model = load_model(model_path)
  except ValueError:
    print("The model {} was not loaded correctly".format(name))
    continue
    
  predictions = net_model.predict(test_x_token)
  predictions = np.array([[0 if value < 0.5 else 1 for value in prediction] for prediction in predictions])
  predictions_list.append(predictions)
  # measuring performance on test set
  cr=classification_report(test_y, predictions, target_names=CATEGORIES.values)
  print(cr)

  # Release memory
  K.clear_session()


#**Generating output in XML format**



In [0]:
import xml.etree.ElementTree
import os
NOT='not met'
MET='met'

#gets a idFile, a dictionary with the predictions (category, label) and the name of the classifier used.
def outputToXML(idFile, dictPred, classifier):
    
    path=SST_HOME+'data/test/xml/'+idFile+'.xml'
    output_dir = SST_HOME+'data/output/'+classifier
    
    output = output_dir+'/'+idFile+'.xml'

    if not os.path.exists(output_dir):
      print("\n\nmaking dir")
      s = os.makedirs(output_dir)
      print(output_dir)
      print(s)      
   
    et = xml.etree.ElementTree.parse(path)

    new_tag = xml.etree.ElementTree.SubElement(et.getroot(), 'TAGS')
    
    for cat in dictPred.keys():
        element = xml.etree.ElementTree.SubElement(new_tag, cat)    
        if dictPred[cat]==0:
            element.attrib['met'] = NOT 
        else:
            element.attrib['met'] = MET

    et.write(output)

#function for creating a dictionary with the categories with values 0 or 1
def iniDictPred(labels, categories):
    
    if len(labels)!=len(categories):
        print('Warning!!!')
        return None
    
    dictPred={}
    i=0
    
    for x in categories:
        dictPred[x]=labels[i]
        i=i+1
    return dictPred

In [0]:
#creates the output xml files per training
for i, predictions in enumerate(predictions_list):
  dictionary = zip(IDFILES, predictions)
  for obj in dictionary:
      idFile=str(obj[0][0])
      labels=obj[1] #gets their predictions for this file
      dictPred=iniDictPred(labels, CATEGORIES) #creates a dictionary to join categories and labels 0,1
      outputToXML(idFile,dictPred,"CNN/{}".format(network_names[i]))

  print('output xml files were generated!')

In [0]:
# Writes the output in a file for each training.
for i, name in enumerate(network_names):
  print(name) 
  path = "data/output/CNN/"
  n = path + "{}".format(name)
  os.system('cd "{}" && echo "{}" {} {}results.txt'.format(SST_HOME, name, ">" if i == 0 else ">>", path))
  os.system('cd "{}" && python3 track1_eval.py data/test/gold {} >> {}results.txt'.format(SST_HOME, n, path))